In [1]:
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pickle
import tqdm
import random

from renderer.threecircles import ThreeCircles
from experiments.playconf.environment import Environment, initialize_scene_from_configuration
from experiments.playconf.rl import get_actor, get_experience, get_critic, get_joint, get_random_experience, get_naive_adversarial_loss

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Linux (64bit) [/home/shreyask/anaconda3/envs/tensorflow2/lib/python3.7/site-packages/pymunk/libchipmunk.so]


In [2]:
model = tf.keras.models.load_model('model_zoo/npe_munk1.h5')

In [3]:
env = Environment(model, episodes=3, batch_size=4096)

In [38]:
rewards, configurations, actual, predicted = get_random_experience(env)

In [44]:
actual.shape

(4096, 3, 3, 2)

In [33]:
losses = []

opt = tf.keras.optimizers.RMSprop(learning_rate=0.0003)
model.compile(loss='mse', optimizer=opt)

for iteration in range(100):
    configs = get_naive_adversarial_loss(env)

In [34]:
configs.shape

(1024, 12)

In [4]:
actor = get_actor()
critic = get_critic(num_layers=6)

In [6]:
joint = get_joint(actor, critic)

def loss(y_true, y_pred):
    return -1 * y_pred

for iteration in range(100):
    latents, x, y = get_experience(actor, env)
    
    critic.trainable = True
    critic.compile(loss='mse', optimizer='adam')
    critic_history = critic.fit(x, 100.0 * y, epochs=1, batch_size=64, verbose=1)
    
#     critic.trainable = False
#     critic.compile(loss='mse', optimizer='adam')
#     joint = get_joint(actor, critic)
#     critic.trainable = False
#     joint.compile(loss=loss, optimizer='adam')
#     joint_history = joint.fit(latents, y, epochs=10, batch_size=64, verbose=0)
    
    #print('Critic', critic_history.history['loss'][-1], 'Actor', -1 * joint_history.history['loss'][-1], 'Actual', np.mean(y))

Train on 4096 samples
4096/4096 [==============================] - 1s 130us/sample - loss: 0.4897
Train on 4096 samples
4096/4096 [==============================] - 1s 128us/sample - loss: 0.2501
Train on 4096 samples
4096/4096 [==============================] - 1s 136us/sample - loss: 0.1640
Train on 4096 samples
4096/4096 [==============================] - 1s 133us/sample - loss: 0.1402
Train on 4096 samples
4096/4096 [==============================] - 1s 135us/sample - loss: 0.1352
Train on 4096 samples
4096/4096 [==============================] - 1s 130us/sample - loss: 0.1345
Train on 4096 samples
4096/4096 [==============================] - 1s 126us/sample - loss: 0.1325
Train on 4096 samples
4096/4096 [==============================] - 1s 131us/sample - loss: 0.1099
Train on 4096 samples
4096/4096 [==============================] - 1s 202us/sample - loss: 0.1207
Train on 4096 samples
4096/4096 [==============================] - 1s 139us/sample - loss: 0.1035
Train on 4096 sample

KeyboardInterrupt: 

In [ ]:
# ES
population_size = 64
sigma = 0.1
learning_rate = 0.01
num_iterations = 500

config_dim = 4*3

guess = np.random.uniform(0.0, 1.0, size=(config_dim,))

pbar = tqdm.tqdm_notebook(range(num_iterations))

scene = ThreeCircles(headless=False)

for iteration in pbar:
    # Create a population sized vectors of different random noise injected into
    # our current best guess.
    
    initialize_scene_from_configuration(scene, guess)
    scene.draw()
    
    noise = np.random.randn(population_size, config_dim)
    new_guesses = np.tile(guess, (population_size, 1)) + (sigma * noise)
    
    R, _, _ = env.get_reward(new_guesses)
    pbar.set_description(str(np.mean(R)))
    
    A = (R - np.mean(R)) / np.std(R)
    guess = guess + learning_rate/(population_size*sigma) * np.dot(noise.T, A)

In [37]:
latents

array([[0.62148099, 0.86766739, 0.80100039, ..., 0.88053498, 0.87362255,
        0.45707031],
       [0.25321808, 0.25814701, 0.05549942, ..., 0.23555466, 0.13853288,
        0.72922545],
       [0.47594484, 0.99991343, 0.0407273 , ..., 0.82290927, 0.88330758,
        0.75143949],
       ...,
       [0.39644988, 0.50016814, 0.16107032, ..., 0.8378394 , 0.76679267,
        0.7031461 ],
       [0.3898526 , 0.59454427, 0.30499382, ..., 0.10895974, 0.46956904,
        0.84048126],
       [0.98441122, 0.51602657, 0.63113396, ..., 0.77286999, 0.45761765,
        0.82785517]])

In [14]:
A = np.array([[1.0, 0.5, 1., 1., 0, 0, 1.`, 1., 0.08, 0.9, 1., 1.]])
pp = critic.predict(A)
env2 = Environment(model, episodes=3, batch_size=1)
r, a, p = env2.get_reward(A)

In [15]:
r, pp/100.0

(array([0.01745741], dtype=float32), array([[0.1650574]], dtype=float32))

In [37]:
for c in configs:
    initialize_scene_from_configuration(scene, c)
    scene.draw()

In [143]:
scene.step()
scene.draw()